In [1]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('../data/seq_0.csv', index_col=0)

In [78]:
data.tail()

,9,10,11,12,13,14,15,16,17,18,...,21,22,23,moving,n_entry_nodes,n_exit_nodes,n_core_nodes,n_paths,congestion,nb_saturated
496,0.2,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.333333,0.500,...,0.8,0.0,0.0,4,3,6,15,5,0.179932,0
497,0.0,0.0,0.0,0.0,0.2,0.5,0.0,0.0,0.500000,0.500,...,0.5,0.0,0.0,4,3,6,15,5,0.197959,0
498,0.2,0.0,0.0,0.0,0.1,0.6,0.0,0.0,0.500000,0.125,...,0.8,0.0,0.0,4,3,6,15,5,0.176276,0
499,0.1,0.0,0.0,0.0,0.2,0.4,0.0,0.0,0.333333,0.625,...,0.7,0.0,0.0,4,3,6,15,5,0.209269,0
500,0.2,0.0,0.0,0.0,0.3,0.9,0.0,0.0,0.166667,0.125,...,0.3,0.0,0.0,4,3,6,15,5,0.193282,0


### Sequence preparation

In [17]:
lookback = 24 # consider the past 24 hours
steps = 6 # sample of 6  data points
delay = 4 # predict in 4 hours
n_nodes = int(data.iloc[0,:].n_core_nodes) # last sample dim

In [35]:
# sample size
samples = np.zeros((len(data) - delay - 1, steps, n_nodes))

In [36]:
samples.shape

(496, 6, 15)

In [37]:
# target size
target = np.zeros((len(data) - delay - 1, ))

In [38]:
target.shape

(496,)

In [89]:
for i in range(len(data)-lookback-delay):
    start = i
    end = i + lookback
    samples[i,:,:] = data.iloc[start:end:lookback//steps, :n_nodes].as_matrix()
    target[i] = data.congestion[end+delay]

In [95]:
samples[0]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.1       ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.3       ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.1       ,
         0.3       ,  0.        ,  0.        ,  0.        ,  0.125     ,
         0.        ,  0.14285714,  0.3       ,  0.        ,  0.        ],
       [ 0.1       ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.3       ,  0.        ,  0.        ,  0.33333333,  0.25      ,
         0.        ,  0.28571429,  0.3       ,  0.        ,  0.        ],
       [ 0.2       ,  0.        ,  0.        ,  0.        ,  0.4       ,
         0.4       ,  0.        ,  0.        , 

In [94]:
target[0]

0.048214285714285716

In [96]:
x_train = samples[:400]
x_test = samples[400:]
y_train = target[:400]
y_test = target[400:]

In [97]:
np.save('../data/serialized_seq/x_train', x_train)
np.save('../data/serialized_seq/y_train', y_train)
np.save('../data/serialized_seq/x_test', x_test)
np.save('../data/serialized_seq/y_test', y_test)
